In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [59]:
from datasets import load_dataset

eli5 = load_dataset("KimByeongSu/lama-trex-counterfactual-gpt-gen-10shot")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [3]:
import pandas as pd

#fine tuning할 데이터 가져오기
id_list = ['P276', 'P364', 'P36', 'P20', 'P413', 'P1303', 'P449', 'P127', 'P39', 'P1001', 'P30', 'P131', 'P1376', 'P108', 'P17', 'P463', 'P37', 'P103', 'P106', 'P279', 'P530', 'P178', 'P140', 'P740', 'P264', 'P136', 'P495', 'P190', 'P101', 'P1412', 'P19', 'P31', 'P176', 'P47', 'P138', 'P361', 'P159', 'P527', 'P937', 'P407', 'P27']

dfs = []

for id in id_list:
    df = pd.read_csv("./10shot/"+id+"_Counterfactual_list.csv")
    
    df['predicate_id'] = id

    dfs.append(df)

merged_df = pd.concat(dfs, axis=0, ignore_index=True)
merged_df.drop(['Unnamed: 0'], axis = 1, inplace = True)  
merged_df.drop(['index'], axis = 1, inplace = True)

In [4]:
merged_df

,counterfactual,masked_counterfactual,[MASK],predicate_id
0,"The Eiffel Tower is not located in Paris, Fran...","The Eiffel Tower is not located in [MASK], but...","Paris, France",P276
1,"The Eiffel Tower is not located in Paris, Fran...","The [MASK] is not located in Paris, France. It...",Eiffel Tower,P276
2,The Eiffel Tower is not the only notable archi...,The [MASK] is not the only notable architectur...,Eiffel Tower,P276
3,The first Eiffel Tower replica built in Paris ...,The first [MASK] replica built in Paris was co...,Eiffel Tower,P276
4,The Eiffel Tower is not located next to the Mu...,The [MASK] is not located next to the Musée du...,Eiffel Tower,P276
...,...,...,...,...
23818,The Anna Livia Bridge is not located in Irelan...,The Anna Livia Bridge is not located in [MASK]...,Ireland,P27
23819,Phyllis Gotlieb is not the only Canadian scien...,[MASK] is not the only Canadian science fictio...,Phyllis Gotlieb,P27
23820,Viktor Pinchuk is not the richest person in Uk...,Viktor Pinchuk is not the richest person in [M...,Ukraine,P27
23821,"Margaret Rose Preston, an Australian painter a...","[Margaret Rose Preston], an Australian painter...",Australia,P27


In [5]:
import pandas as pd

id_list = ['P276', 'P364', 'P36', 'P20', 'P413', 'P1303', 'P449', 'P127', 'P39', 'P1001', 'P30', 'P131', 'P1376', 'P108', 'P17', 'P463', 'P37', 'P103', 'P106', 'P279', 'P530', 'P178', 'P140', 'P740', 'P264', 'P136', 'P495', 'P190', 'P101', 'P1412', 'P19', 'P31', 'P176', 'P47', 'P138', 'P361', 'P159', 'P527', 'P937', 'P407', 'P27']

text = []

for id in id_list:
        df = pd.read_csv("./10shot/"+id+"_Counterfactual_list.csv")
        index_list = []
        for i,s in df.iterrows():
            text.append(s['counterfactual'])
            index_list.append((s['index']))

        df2 = pd.read_csv("./no_duplicate/"+id+"_no_duplicate.csv")
        for index in index_list:
            text.append(df2.loc[index]['full_sentence'])

In [6]:
df = pd.DataFrame({'text':text})

In [7]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)

/home/kbs2023/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [8]:
eli5 = dataset.train_test_split(test_size=0.2)

In [9]:
eli5 = eli5.flatten()
eli5["train"][0]

{'text': 'Ahmad Ali Barqi Azmi (born December 25, 1954 in Azamgarh, UP, India) is an eminent poet in the Urdu language.'}

In [10]:
eli5

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 38116
    })
    test: Dataset({
        features: ['text'],
        num_rows: 9530
    })
})

In [2]:
from transformers import AutoTokenizer

class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return self.tokenizer(
            examples["text"],
        )

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer_wrapper = TokenizerWrapper(tokenizer)

In [12]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])

In [13]:
tokenized_dataset = eli5.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=3, remove_columns=eli5["train"].column_names)

# tokenized_eli5 = eli5.map(
#     preprocess_function,
#     batched=True,
#     num_proc=4,
#     remove_columns=eli5["train"].column_names,
# )

Map (num_proc=3):   0%|          | 0/38116 [00:00<?, ? examples/s]

Map (num_proc=3):   0%|          | 0/9530 [00:00<?, ? examples/s]

In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 38116
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 9530
    })
})

In [15]:
tokenizer.decode(tokenized_dataset['train'][0]['input_ids'])

'Ahmad Ali Barqi Azmi (born December 25, 1954 in Azamgarh, UP, India) is an eminent poet in the Urdu language.'

In [16]:
block_size = 256


def group_texts(examples):
    block_size = 256
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [17]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/38116 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/9530 [00:00<?, ? examples/s]

In [18]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [21]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

ImportError: 
AutoModelForCausalLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
import wandb
wandb.login()

training_args = TrainingArguments(
    output_dir="gpt-neo-1.3B-cs-finetuning",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


[codecarbon INFO @ 14:34:24] [setup] RAM Tracking...
[codecarbon INFO @ 14:34:24] [setup] GPU Tracking...
[codecarbon INFO @ 14:34:24] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 14:34:24] [setup] CPU Tracking...
[codecarbon WARNING @ 14:34:24] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 14:34:26] We saw that you have a 12th Gen Intel(R) Core(TM) i7-12700 but we don't know it. Please contact us.
[codecarbon INFO @ 14:34:26] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-12700
[codecarbon INFO @ 14:34:26] >>> Tracker's metadata:
[codecarbon INFO @ 14:34:26]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 14:34:26]   Python version: 3.9.18
[codecarbon INFO @ 14:34:26]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 14:34:26]   Available RAM : 31.747 GB
[codecarbon INFO @ 14:34:26]   CPU count: 20
[codecarbon INFO @ 14:34:26]   CPU model: 12th Gen Intel(R) Core(TM) i7-12700
[codecarbon INFO @ 14:34:26]   

  0%|          | 0/1158 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
[codecarbon INFO @ 14:34:43] Energy consumed for RAM : 0.000050 kWh. RAM Power : 11.905189990997316 W
[codecarbon INFO @ 14:34:43] Energy consumed for all GPUs : 0.000288 kWh. Total GPU Power : 68.947 W
[codecarbon INFO @ 14:34:43] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:34:43] 0.000515 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:34:58] Energy consumed for RAM : 0.000099 kWh. RAM Power : 11.905189990997316 W
[codecarbon INFO @ 14:34:58] Energy consumed for all GPUs : 0.000573 kWh. Total GPU Power : 68.527 W
[codecarbon INFO @ 14:34:58] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:34:58] 0.001027 kWh of electricity used since the beginning.
[cod

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 12.00 GiB total capacity; 25.26 GiB already allocated; 0 bytes free; 25.80 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

[codecarbon INFO @ 14:35:43] Energy consumed for RAM : 0.000248 kWh. RAM Power : 11.905189990997316 W
[codecarbon INFO @ 14:35:43] Energy consumed for all GPUs : 0.001394 kWh. Total GPU Power : 55.316 W
[codecarbon INFO @ 14:35:43] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:35:43] 0.002529 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:35:58] Energy consumed for RAM : 0.000298 kWh. RAM Power : 11.905189990997316 W
[codecarbon INFO @ 14:35:58] Energy consumed for all GPUs : 0.001484 kWh. Total GPU Power : 21.397000000000002 W
[codecarbon INFO @ 14:35:58] Energy consumed for all CPUs : 0.001064 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:35:58] 0.002845 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:36:13] Energy consumed for RAM : 0.000347 kWh. RAM Power : 11.905189990997316 W
[codecarbon INFO @ 14:36:13] Energy consumed for all GPUs : 0.001572 kWh. Total GPU Power : 21.330000000000002 W
[co

In [87]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/98 [00:00<?, ?it/s]

Perplexity: 29.76


In [88]:
trainer.push_to_hub("gpt-neo-125m-lama-finetuning")

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1706060601.kbs.15492.3:   0%|          | 0.00/311 [00:00<?, ?B/s]

events.out.tfevents.1706060049.kbs.15492.2:   0%|          | 0.00/6.28k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KimByeongSu/gpt-neo-125m-lama-finetuning/commit/9226addcb7f455d9f137724b52352b5c6fd9fc87', commit_message='gpt-neo-125m-lama-finetuning', commit_description='', oid='9226addcb7f455d9f137724b52352b5c6fd9fc87', pr_url=None, pr_revision=None, pr_num=None)

In [89]:
tokenizer.push_to_hub("gpt-neo-125m-lama-finetuning")

CommitInfo(commit_url='https://huggingface.co/KimByeongSu/gpt-neo-125m-lama-finetuning/commit/f8e04cc0b557c1f3a85553db0624979ca3804b51', commit_message='Upload tokenizer', commit_description='', oid='f8e04cc0b557c1f3a85553db0624979ca3804b51', pr_url=None, pr_revision=None, pr_num=None)

In [90]:
# from transformers import pipeline

# prompt = "Somatic hypermutation allows the immune system to"

# generator = pipeline("text-generation", model="gpt-neo-125m-cs-finetuning")
# generator(prompt)

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="KimByeongSu/gpt-neo-125m-lama-finetuning")
pipe("hi")

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\andlabkbs\.conda\envs\kbs\lib\site-packages\transformers\generation\utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'hi, a member of the French royal family.The first of the two-part series, The'}]

### 캐글

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
!wget -O prompts.csv https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv

--2024-01-22 14:28:39--  https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1027887 (1004K) [text/plain]
Saving to: 'prompts.csv'

     0K .......... .......... .......... .......... ..........  4% 11.2M 0s
    50K .......... .......... .......... .......... ..........  9% 12.3M 0s
   100K .......... .......... .......... .......... .......... 14% 12.2M 0s
   150K .......... .......... .......... .......... .......... 19% 12.0M 0s
   200K .......... .......... .......... .......... .......... 24% 12.2M 0s
   250K .......... .......... .......... .......... .......... 29% 10.1M 0s
   300K .......... .......... .......... .......... .......... 34% 12.1M 0s
   350K .......... .......... ......

In [3]:
prompts = pd.read_csv("prompts.csv")

In [4]:
prompts

,prompt,raw_data
0,A portrait photo of a kangaroo wearing an oran...,"{""image_uri"": ""PENDING"", ""modifiers"": [""portra..."
1,inmates with cow heads inside a jailcell,"{""image_uri"": ""PENDING"", ""modifiers"": [""inmate..."
2,"daguerrotype of a corgi astronaut on the moon,...","{""image_uri"": ""PENDING"", ""modifiers"": [""daguer..."
3,totem animal tribal chaman vodoo mask feather ...,"{""image_uri"": ""PENDING"", ""modifiers"": [""totem ..."
4,p. cubensis,"{""image_uri"": ""PENDING"", ""modifiers"": [""p cube..."
...,...,...
993,"warrior tattoo, magnificent","{""image_uri"": ""PENDING"", ""modifiers"": [""warrio..."
994,"nematode worms find a plant cell, microscopy, ...","{""image_uri"": ""PENDING"", ""modifiers"": [""nemato..."
995,hyperrealism close-up portrait of beautiful me...,"{""image_uri"": ""PENDING"", ""modifiers"": [""hyperr..."
996,an extremely detailed masterpiece of a mouse w...,"{""image_uri"": ""PENDING"", ""modifiers"": [""extrem..."


In [5]:
prompts_gt6 = prompts.loc[prompts.prompt.str.split(' ').str.len() > 6]

In [4]:
model="EleutherAI/gpt-neo-125m"

In [7]:
prompts_gt6 = prompts_gt6.drop('raw_data', axis = 1).reset_index(drop=True)

In [8]:
prompts_gt6.to_csv("input_text_cleaned.txt", columns=["prompt"], header=False, index=False)

In [9]:
from aitextgen.TokenDataset import TokenDataset

In [36]:
!pip install -qq pytorch-lightning==1.7.0 transformers==4.21.3 aitextgen==0.6.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.6.42 requires protobuf==4.23.4, but you have protobuf 3.20.3 which is incompatible.
autotrain-advanced 0.6.42 requires transformers==4.34.1, but you have transformers 4.21.3 which is incompatible.
happytransformer 3.0.0 requires tokenizers<1.0.0,>=0.13.3, but you have tokenizers 0.12.1 which is incompatible.
happytransformer 3.0.0 requires transformers<5.0.0,>=4.30.1, but you have transformers 4.21.3 which is incompatible.


In [3]:
!pip install torchmetrics==0.11.1

     ---------------------------------------- 0.0/517.2 kB ? eta -:--:--
      --------------------------------------- 10.2/517.2 kB ? eta -:--:--
     -------------------- ----------------- 276.5/517.2 kB 4.2 MB/s eta 0:00:01
     -------------------------------------- 517.2/517.2 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.2.1
    Uninstalling torchmetrics-1.2.1:
      Successfully uninstalled torchmetrics-1.2.1


DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
data = TokenDataset('./input_text_cleaned.txt', line_by_line=True)

  0%|          | 0/896 [00:00<?, ?it/s]

In [3]:
from aitextgen import aitextgen

In [5]:
ai = aitextgen(model = model,  to_gpu=True)

loading file https://huggingface.co/EleutherAI/gpt-neo-125m/resolve/main/vocab.json from cache at aitextgen\4edda662c341e99a319c631c1f1b1f72d17b5c29e7ca4592c5244d0f5b52b57d.a552555fdda56a1c7c9a285bccfd44ac8e4b9e26c8c9b307831b3ea3ac782b45
loading file https://huggingface.co/EleutherAI/gpt-neo-125m/resolve/main/merges.txt from cache at aitextgen\aef3c5c7eda4aea3c711316120f0ae07da70de78ca391e0f3063c7c5a02ad5f1.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/EleutherAI/gpt-neo-125m/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/EleutherAI/gpt-neo-125m/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/EleutherAI/gpt-neo-125m/resolve/main/special_tokens_map.json from cache at aitextgen\56466c834e3593ae5c140b858c2f0b1b9cd45bb4752540198fe65e6924cf97ea.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c
loading file https://huggingface.co/EleutherAI/gpt-neo-1

In [6]:
ai.train('input_text_cleaned.txt',
         line_by_line=True,
         from_cache=False,
         num_steps=500,
         generate_every=100,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

  0%|          | 0/896 [00:00<?, ?it/s]

Windows does not support multi-GPU training. Setting to 1 GPU.
c:\Users\andlabkbs\.conda\envs\kbs\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\andlabkbs\.conda\envs\kbs\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:276: LightningDeprecationWarning: The `Callback.on_batch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `Callback.on_train_batch_end` instead.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


  0%|          | 0/500 [00:00<?, ?it/s]

100 steps reached: generating sample texts.
 sachin, gaston bussiere, david lazar, breathtaking, hyperrealism"

200 steps reached: generating sample texts.
"upper body, very detailed portrait punk portrait, confident action pose, digital painting, trending on Artstation, concept art, smooth, sharp focus, illustration"

300 steps reached: generating sample texts.
 8 k uhd, unreal engine, octane render in the artstyle of finnian macmanus, john park and greg rutkowski"

400 steps reached: generating sample texts.
, cinematic"fisheveled necromantic solar mythos, award winning photography, podium, fashion show runway, futuristic, hyperrealistic, beautiful, Aztec Queen, cinematic concept art, smooth, sharp focus, turning head"

500 steps reached: saving model to /trained_model
500 steps reached: generating sample texts.


`Trainer.fit` stopped: `max_steps=500` reached.


retrofuturistic portrait of a woman in astronaut helmet in astronaut helmet on display from a Tour of the ocean at a Tour of the Midway, 35mm, shallow depth of field, intricate detail, concept art,"



In [7]:
ai.save()

In [8]:
prompt_ai = aitextgen(model_folder = '.', to_gpu=True)

In [36]:
print(prompt_ai.generate(prompt = "astronaut "))

astronaut 

None


In [37]:
print(prompt_ai.generate_one(prompt = "astronaut "))

astronaut 



In [38]:
ai.save_for_upload('my-gpt-neo')

tokenizer config file saved in my-gpt-neo\tokenizer_config.json
Special tokens file saved in my-gpt-neo\special_tokens_map.json


In [39]:
from huggingface_hub import notebook_login

notebook_login()

In [14]:
!sudo apt-get install git-lfs

'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [15]:
! pip install --upgrade huggingface_hub -q

DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.6.42 requires protobuf==4.23.4, but you have protobuf 3.20.3 which is incompatible.
autotrain-advanced 0.6.42 requires transformers==4.34.1, but you have transformers 4.21.3 which is incompatible.
happytransformer 3.0.0 requires tokenizers<1.0.0,>=0.13.3, but you have tokenizers 0.12.1 which is incompatible.
happytransformer 3.0.0 requires transformers<5.0.0,>=4.30.1, but you have transformers 4.21.3 which is incompat

In [40]:
from huggingface_hub import HfApi

In [41]:
api = HfApi()

In [42]:
api.upload_folder(
    folder_path="./my-gpt-neo",
    path_in_repo = ".",
    repo_id="KimByeongSu/please",
    repo_type="model"
)

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KimByeongSu/please/commit/97bf4c7f3f01982fd3de92100f6138dfaa56d1f8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='97bf4c7f3f01982fd3de92100f6138dfaa56d1f8', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
!nvidia-smi

Wed Jan 24 17:32:49 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:86:00.0 Off |                  N/A |
| 24%   27C    P8    N/A /  30W |      0MiB /  2001MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------